<a href="https://colab.research.google.com/github/cs-amy/project-codebase/blob/main/Character_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model for Letter De-Obfuscation**
Stage 1 of MSc Project - Ashraf Muhammed Yusuf

# **1. Colab Environment Setup**
Open a this notebook in Google Colab and set runtime to TPU / GPU / CPU

In [ ]:
# Mount your Drive so you can read data/characters and write checkpoints
# Link to dataset:
# https://drive.google.com/drive/folders/1sfNG1PkmTPBe1wOSQXZmfdkvR97Hn9lk?usp=sharing
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# Install dependencies
!pip install -q tensorflow matplotlib

In [3]:
# Import dependencies
import os
import glob
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, mixed_precision
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
# (Optional—but useful) turn on XLA JIT for extra speed
tf.config.optimizer.set_jit(True)

# **2. Data Loading & Splitting**
We'll use Keras's image_dataset_from_directory to build train/validation and test sets.

In [ ]:
BATCH = 64 if tf.config.list_physical_devices('TPU') else 32
IMG_SIZE = (64, 64)
train_dir = "/content/drive/MyDrive/MScProject/data/characters/train"
test_dir = "/content/drive/MyDrive/MScProject/data/characters/test"

def preprocess(img, label):
  img = tf.image.convert_image_dtype(img, tf.float32) # 0–1
  img = tf.image.resize(img, [64, 64])
  return img, label



# Quick .png count per class
print("Train .png counts:")
for cls in sorted(os.listdir(train_dir)):
  pattern = os.path.join(train_dir, cls, '*.png')
  print(f"  {cls}: {len(glob.glob(pattern))}")

print("\nTest .png counts:")
for cls in sorted(os.listdir(test_dir)):
  pattern = os.path.join(test_dir, cls, '*.png')
  print(f"  {cls}: {len(glob.glob(pattern))}")

# Train dataset
train_ds = (
  tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH,
    image_size=IMG_SIZE,
    color_mode="grayscale",
    validation_split=0.20,
    subset="training",
    seed=42,
  )
)


# Val dataset
val_ds = (
  tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH,
    image_size=IMG_SIZE,
    color_mode="grayscale",
    validation_split=0.20,
    subset="validation",
    seed=42,
  )
)

In [32]:
# Test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SIZE,
  color_mode="grayscale",
  shuffle=False,
)

Found 7655 files belonging to 26 classes.


In [ ]:
# Utility to display examples from each set
def show_examples(ds, ds_name, num=5):
  # Take one batch
  for images, labels in ds.take(1):
    images = images.numpy()
    labels = labels.numpy()
    class_names = ds.class_names
    break

  plt.figure(figsize=(6,6))
  for i in range(num):
    ax = plt.subplot(3, 3, i+1)
    img = images[i].squeeze()  # shape: (H,W) since grayscale
    lbl = class_names[labels[i].argmax()]
    plt.imshow(img, cmap='gray')
    plt.title(f"{ds_name}: {lbl}")
    plt.axis('off')
  plt.tight_layout()
  plt.show()

# Display 5 examples from each split
show_examples(train_ds, "Train")
show_examples(val_ds,   "Val")
show_examples(test_ds,  "Test")

# **3. Preprocessing & Augmentation**

In [33]:
# Save dataset class names before piping the dataset through 'map'
train_ds_class_names = test_ds.class_names
val_ds_class_names = test_ds.class_names
test_ds_class_names = test_ds.class_names

In [34]:
# Normalize and augment datasets (only the train dataset is augmented)
normalization = layers.Rescaling(1./255)
aug = tf.keras.Sequential([
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
  layers.RandomTranslation(0.1,0.1)
])

train_ds = train_ds.map(lambda x,y: (aug(normalization(x)), y))
val_ds   = val_ds.map(lambda x,y: (normalization(x), y))
test_ds  = test_ds.map(lambda x,y: (normalization(x), y))

# **4. Model Architecture**
We will define a simple yet robust CNN (grayscale)

In [ ]:
# Define model:
# 1. Input Layer
inputs = layers.Input(shape=(*IMG_SIZE, 1))
# 2. First Convolution + Pooling
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D()(x)
# 3. Second Convolution + Pooling
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
# 4. Third Convolution
x = layers.Conv2D(128, 3, activation='relu')(x)
# 5. Flatten to Vector
x = layers.Flatten()(x)
# 6. Fully‑Connected (Dense) Layer
x = layers.Dense(128, activation='relu')(x)
# 7. Dropout Regularisation
x = layers.Dropout(0.5)(x)
# 8. Output Layer
outputs = layers.Dense(26, activation='softmax')(x)

# Construct model
model = models.Model(inputs, outputs)

# Show model summary
model.summary()

# **5. Compilation & Callbacks**

In [17]:
# Compile model
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Callbacks
# 1. Checkpoint - saves the best model
ckpt = callbacks.ModelCheckpoint(
  filepath="/content/drive/MyDrive/MScProject/char_cnn_ckpt_best.keras",
  save_best_only=True,
  monitor="val_loss" # keeps only the best model
)
# 2. Early stopping
es = callbacks.EarlyStopping(
  monitor="val_loss",
  patience=6, # stop ~6 epochs after val_loss stalls
  restore_best_weights=True
)

# 3. Learning rate scheduler
scheduler = callbacks.ReduceLROnPlateau(
  monitor="val_loss",
  factor=0.5,
  patience=3, # halve LR if val_loss hasn’t improved for 3 epochs
  min_lr=1e-6
)

callbacks=[ckpt, es, scheduler]

# **6. Training**

In [ ]:
# Train model
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=50,
  callbacks=callbacks
)

# **7. Evaluation**

In [ ]:
# Test model accuracy on test dataset
model.evaluate(test_ds)

# **Confusion Matrix & Classification Report**

In [ ]:
# Gather all ground-truths and predictions
y_true = []
y_pred = []
for batch_x, batch_y in test_ds:
  preds = model.predict(batch_x)
  y_pred.extend(np.argmax(preds, axis=1))
  y_true.extend(np.argmax(batch_y.numpy(), axis=1))

class_names = test_ds_class_names
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)

# **Visualization**

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=90)
plt.yticks(tick_marks, class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

In [ ]:
# Training curves
epochs = range(1, len(history.history['loss']) + 1)
plt.figure(figsize=(12, 4))

In [ ]:
# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['accuracy'],    label='train_acc')
plt.plot(epochs, history.history['val_accuracy'],label='val_acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

In [ ]:
# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['loss'],    label='train_loss')
plt.plot(epochs, history.history['val_loss'],label='val_loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()

# **9. Display Misclassified Examples**

In [ ]:
# First flatten all test images to a single array
images_all = np.concatenate([x.numpy() for x, y in test_ds], axis=0)
mis_idx = [i for i, (t, p) in enumerate(zip(y_true, y_pred)) if t!=p]

plt.figure(figsize=(9,9))
for i, idx in enumerate(mis_idx[:9]):
  plt.subplot(3, 3, i+1)
  img = images_all[idx].squeeze()
  plt.imshow(img, cmap='gray')
  plt.title(f"T:{class_names[y_true[idx]]} P:{class_names[y_pred[idx]]}")
  plt.axis('off')
plt.tight_layout()
plt.show()

# **8. Save & Document**

In [48]:
# (Optional) Export label map (a - z)
import json
with open("/content/drive/MyDrive/MScProject/label_map.json","w") as f:
    json.dump(train_ds_class_names, f)